In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from shapely.wkt import loads 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# MGRS/UTM Grid
`civicknowledge.com-mgrs-1.1.3` Last Update: 2021-02-09T21:31:43

_Military Grid Reference System grid polygons_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [utm_grid](notebooks/Extract.ipynb#s)**. UTM Grid polygons, with links to US states

## References
<ul><li> <strong>utm_grid_source</string>, <em>data/utm-grid.csv</em>. </li><li> <strong>geohash_pkg</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv</em>. </li><li> <strong>us_states</string>, <em>censusgeo://2019/5/US/state</em>. </li><ul>

In [5]:
s = pkg.reference('utm_grid_source').dataframe()
s.columns = ['geometry','fid','band','row','west_value','cm_value','east_value']
s['geometry'] = s.geometry.apply(loads)
s = s[s.west_value != 'NoZN']
s = gpd.GeoDataFrame(s, geometry='geometry', crs=4326)
s['zone'] = s.apply(lambda r: f"{r.band:02d}{r.row}", axis=1)
s = s.drop(columns=['fid']).sort_values('zone')

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    s.centroid.y

def _epsg(r):
    if r.geometry.centroid.y >= 0:
        return f"326{r.band:02d}"
    else:
        return f"327{r.band:02d}"
        
    
s['epsg'] = s.apply(_epsg, axis=1).astype(int)


In [ ]:
s.epsg.unique()

In [ ]:
states = pkg.reference('us_states').geoframe().to_crs(4326)
continental_states = states[~states.stusps.isin(['HI','AK','PR','VI','MP','GU','AS'])]
continental_states.plot(figsize=(10,10))
us_boundary = continental_states.unary_union

In [ ]:
t = gpd.sjoin(s, states )
state_zones = t[['zone', 'stusps']].drop_duplicates().zone

t = gpd.sjoin(s, continental_states )
cstate_zones = t[['zone', 'stusps']].drop_duplicates().zone

s['us_state'] = (s.zone.isin(state_zones)).astype(int)
s['cus_state'] = (s.zone.isin(cstate_zones)).astype(int)

In [ ]:

s.plot(column='cus_state')

In [ ]:
s.head()